#### Set Up

In [4]:
from pathlib import Path
import json

CLASSIFICATIONS = {
    0: 'none',
    1: 'plot',
    2: 'remove_duplicate_values',
    3: 'fill_missing_values',
    4: 'perform_dimensionality_reduction',
    5: 'perform_correalation_analysis',
}

system_prompts = {f.stem: f.read_text() for f in Path("system_prompt").glob("*.txt")}
system_prompt_classification = system_prompts['classification'].format(functions_dict=str(CLASSIFICATIONS))

user_prompts = [f.read_text() for f in Path("user_prompt").glob("*.txt")]
structured_outputs = {f.stem: json.load(f.open('r')) for f in Path("structured_output").glob("*.json")}

#### Request Classification & Structure Output

In [5]:
import re

def postprocess_classification_respond(respond):
    match = re.search(r'\d+', respond)
    if match:
        res = int(match.group(0))
        if res < len(CLASSIFICATIONS) and res > 0:
            return res
    return 0

def perform_classification(client, user_request):
    messages = [
        {
            "role": "system",
            "content": system_prompt_classification,
        },
        {
            "role": "user",
            "content": user_request,
        }
    ]
    response = client.chat.completions.create(
        model="solar-pro",
        messages=messages,
    )
    return postprocess_classification_respond(response.choices[0].message.content)

def perform_function_mapping(client, user_request, system_prompt, structured_output):
    messages = [
            {
                'role': 'system',
                'content': system_prompt
            },
            {
                'role': 'user',
                'content': user_request
            }
        ]
    response = client.chat.completions.create(
            model="solar-pro",
            messages=messages,
            response_format=structured_output
        )
    return response.choices[0].message.content

def process_user_request(client, user_request):
    '''
    Outpupt: function_name (string), function_parameters (JSON), model_respond (string)
    Return the JSON string with the information of the function that the user want to call together the respond of the system.
    '''
    classification = perform_classification(client, user_request)
    function_name = CLASSIFICATIONS[classification]
    function_parameters = {}
    
    if (classification == 2):
        function_parameters = perform_function_mapping(
                client, user_request, 
                system_prompt = system_prompts[CLASSIFICATIONS[classification]], 
                structured_output = structured_outputs[CLASSIFICATIONS[classification]])
        try:
            function_parameters = json.loads(function_parameters)
        except Exception as e:
            function_parameters = {}
            
    return function_name, function_parameters

In [6]:
from openai import OpenAI
import toml

parsed_toml = toml.load('../../../secrets.toml')

client = OpenAI(
	api_key=parsed_toml['upstage_api_key'], 
 	base_url="https://api.upstage.ai/v1"
)

for user_prompt in user_prompts:
    function_name, function_parameters = process_user_request(client, user_request=user_prompt)
    print(function_name)
    print(function_parameters)

InternalServerError: Error code: 500 - {'error': {'message': 'internal_server_error', 'type': 'internal_server_error', 'param': None, 'code': None}}